# SQL II: Grouping and Joining

_June 29, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT * FROM tables GROUP BY column`

In [2]:
# instantiate some data
np.random.seed(1000)
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Eric R","Dolci","Jagandeep"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Eric A","Eric R","Eryk W"],
                            "quiz_score":np.random.randint(0,10,3)})

In [3]:
conn = sqlite3.connect('lecture_code.db')
c = conn.cursor()

In [4]:
# insert these datasets into our db
#auto.to_sql('auto', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

In [6]:
#get a list of all tables
query = """SELECT name FROM sqlite_master WHERE type = 'table' 
ORDER BY name"""
c.execute(query)
c.fetchall()

[('small_grade',), ('small_quiz',), ('tips',)]

#### TIPS dataset

In [14]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [13]:
# for the tips df/table, use PANDAS to get a count of a total of men and women
tips.sex.value_counts()
tips.groupby(['sex']).size()

sex
Male      157
Female     87
dtype: int64

In [18]:
# translate this to sql 
query = """SELECT sex, COUNT(*) FROM tips GROUP BY sex"""
c.execute(query).fetchall()

[('Female', 87), ('Male', 157)]

In [25]:
# using PANDAS, select the average amount tipped for time(lunch and dinner)
tips.groupby(['time']).tip.mean()

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [28]:
# use sql to recreate this query
query = """SELECT time, AVG(tip) FROM tips GROUP BY time ORDER BY AVG(tip) ASC"""
c.execute(query).fetchall()

[('Lunch', 2.7280882352941176), ('Dinner', 3.102670454545454)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

In [41]:
# use pandas, get the average total bills for a party larger than 2 for every size of party
tips.groupby(['size']).total_bill.mean()[2:]


size
3    23.277632
4    28.613514
5    30.068000
6    34.830000
Name: total_bill, dtype: float64

In [44]:
# use sql to recreate this query hint: use having or where
query = """SELECT size, AVG(total_bill) 
FROM tips
GROUP BY size
HAVING size > 2
"""
c.execute(query).fetchall()

[(3, 23.27763157894737), (4, 28.61351351351351), (5, 30.068), (6, 34.83)]

In [112]:
# exercise 1 - use pandas, get the maximum value of total bill for female non smoker
#tips.groupby(['sex', 'smoker'])['total_bill'].max()[3:]
tips[(tips.sex == 'Female') & (tips.smoker == 'No')].total_bill.max()

35.83

In [113]:
# use sql to recreate this query
query = """SELECT sex, smoker, MAX(total_bill) AS max_bill
FROM tips
WHERE sex = 'Female' AND smoker = 'No'
"""
c.execute(query).fetchall()

[('Female', 'No', 35.83)]

In [105]:
# exercise 2 - use PANDAS, the average amount of tips and total bill for different day of the weak 
# and sort by descending value of tip amount
tips.groupby(['day'])['tip', 'total_bill'].mean().sort_values(by = 'tip', ascending = False)

,tip,total_bill
day,,
Sun,3.255132,21.410000
Sat,2.993103,20.441379
Thur,2.771452,17.682742
Fri,2.734737,17.151579


In [106]:
# translate this to sql
query = """SELECT day, AVG(tip) AS avg_tip, AVG(total_bill) AS avg_bill
FROM tips
GROUP BY day
ORDER BY avg_tip DESC"""
c.execute(query).fetchall()

[('Sun', 3.255131578947369, 21.410000000000004),
 ('Sat', 2.993103448275862, 20.441379310344825),
 ('Thur', 2.771451612903226, 17.682741935483865),
 ('Fri', 2.734736842105263, 17.151578947368417)]

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [116]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0   Eric A           1
1   Eric R           0
2   Eryk W           9
    students  projects  grades
0     Eric R         1      99
1      Dolci         2      87
2  Jagandeep         1      80


In [117]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Eric A,1,NaN,NaN
1,Eric R,0,1.0,99.0
2,Eryk W,9,NaN,NaN


Syntax:


`SELECT * FROM left table JOIN right table ON left_table.key = right_table.key`

In [118]:
# sql 
query = """SELECT * FROM small_grade 
        LEFT JOIN small_quiz 
        ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(0, 'Eric A', 1, None, None, None, None),
 (1, 'Eric R', 0, 0, 'Eric R', 1, 99),
 (2, 'Eryk W', 9, None, None, None, None)]

In [ ]:
# investigate why we have more columns than pd results
# keeps duplicates and treat index as columns


In [119]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students', how = 'inner')

,students,quiz_score,projects,grades
0,Eric R,0,1,99


In [121]:
# SQL

query = """SELECT * 
            FROM small_grade
            INNER JOIN small_quiz
            USING (students)"""
c.execute(query).fetchall()

[(1, 'Eric R', 0, 0, 1, 99)]

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""